In [1]:
# Get GenreCounts
# A is empty list to get value counts of Genres
def genre_count(CleanedDF):
    genres = ['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','Foreign',
          'History','Horror','Music','Mystery','Romance','Science Fiction','Thriller','TV Movie','War','Western']
    A = []
    for genre in genres:
        A.append(CleanedDF[genre].value_counts().unique().tolist())
    A 

# Dividing A into flat_list to get the False and True values separated into list
    flat_list = []
    for sublist in A:
        for item in sublist:
            flat_list.append(item)
    flat_list
    length = len(flat_list)
    flat_list

# Creating Flat_list2 to get only the True values which are odd indexes
    flat_list2 = []
    flatlistindex = 0
    for item in flat_list:
        if(flatlistindex)%2!=0:
            flat_list2.append(item)
        flatlistindex += 1

#flat_list2
# Count total of genres
    TotalGenreCount = sum(flat_list2)
    TotalGenreCount

    GenrePercentage = []
    for genrecount in flat_list2:
        GenrePercentage.append(round(genrecount*100/TotalGenreCount,2))
    

    GenreCountsDF = pd.DataFrame({"Genre":genres,
                                  "Counts":flat_list2,
                                  "Percentage":GenrePercentage})      
# Display sorted list of Genres
    GenreCountsDF.sort_values(by = ['Counts'], ascending = False)
    print(GenreCountsDF)
    
# Plot Genre counts as percentage of movies in sample of 5000 movies
    SortedGenreCountsDF = GenreCountsDF.sort_values(by=['Counts'],ascending=False)
#plt.pie(flat_list2,labels= genres, autopct = "%1.1f%%", shadow = True, startangle = 140)
    Top5GenreCountsDF = SortedGenreCountsDF.reset_index()
    Top5GenresDF = Top5GenreCountsDF.loc[0:4,]
    Top5Genres = Top5GenresDF['Genre']
    Top5GenreCounts = Top5GenresDF['Counts']

    Top5GenresDF
# Drama, Comedy, Thriller, Action, Romance
    explode = (0.1,0,0,0,0)

# Plot the top 5 Genres as a Pie Chart
    plt.pie(Top5GenreCounts,labels= Top5Genres, autopct = "%1.1f%%", explode = explode, shadow = True, startangle = 140)
    plt.savefig("Output/Top5GenrePie.png")

In [3]:
def revenue_decade(CleanedDF):
    Clean2DF = CleanedDF[(CleanedDF.budget != 0) & (CleanedDF.revenue != 0)]
# Total 3229 rows
    Clean2DF

# Group by Decade
    DecadeGroup = Clean2DF[['Decade','id','budget','popularity','revenue']].groupby('Decade')

# Count movies in each decade
    MoviesPerDecade = DecadeGroup['id'].count()
    MoviesPerDecade = MoviesPerDecade.reset_index()
    MoviesPerDecade = MoviesPerDecade.rename(columns={"Decade":"Decade","id":"Movie Count"})
    MoviesPerDecade
    
    # Get Revenue / Budget ratio
    RevenueBudgetRatio = DecadeGroup[['budget','revenue']].sum()
    RevenueBudgetRatio['profitratio'] = round(RevenueBudgetRatio['revenue'] / RevenueBudgetRatio['budget'],2)
    RevenueBudgetRatio = RevenueBudgetRatio.reset_index()
    print(RevenueBudgetRatio)
    
    
# Popularity in each decade
    PopularityPerDecade = round(DecadeGroup['popularity'].mean(),2)
    PopularityPerDecade = PopularityPerDecade.reset_index()
    PopularityPerDecade
    
    # Bar Plot
# Obtain the x and y coordinates 
# X Axis is Decade
    from matplotlib.pyplot import text

    Decades = MoviesPerDecade['Decade']
    xAxis = Decades
    yAxis = RevenueBudgetRatio['profitratio']

# Create a bar chart based upon the above data
    ax = plt.bar(xAxis, yAxis, color=(0.5, 0, 0.6, 0.5), align="center")

# Create the ticks for our bar chart's x axis
    tick_locations = [value for value in xAxis]
    plt.xticks(tick_locations, xAxis)

# Incorporate the other graph properties
    plt.xlabel('Decade')
    plt.ylabel('Revenue / Budget')
    plt.title('Revenue-Budget Ratio per Decade')

    plt.ylim(0,max(yAxis)+1)

# Save Figure
    plt.grid()
    plt.tight_layout()

    plt.savefig("output/RevenueBudgetRatioBarPlot.png")
    plt.show()

In [4]:
def profit_runtime(CleanedDF):
    # Condition to remove 0 budget and 0 revenue rows
    Clean2DF = CleanedDF[(CleanedDF.budget != 0) & (CleanedDF.revenue != 0)]
    
# Locate movies with original_language!= 'en'
#CleanedDF.head()
#ForeignLangDF = CleanedDF.loc[(CleanedDF['Foreign']==True)]
#ForeignLangDF

#Clean2DF

# Plot Revenue vs Runtime for Decades
    Clean2DF['runtime'].max()

    runtime_bins = [30,60,90,120,150,180,360]
    runtime_labels = ["30-60","60-90","90-120","120-150","150-180","180-360"]
    Clean2DF['Runtime Range'] = pd.cut(Clean2DF['runtime'], runtime_bins, labels=runtime_labels)

# Group by runtime and choose relevant columns
    RuntimeGroup = Clean2DF[["budget","popularity","revenue","Decade","Runtime Range"]].groupby('Runtime Range')

# Get a count of movies in runtime bins
    RuntimeMovieCount = RuntimeGroup['budget'].count()
    RuntimeMovieCount = RuntimeMovieCount.reset_index()

# Get a mean of revenue
    RuntimeRangeDF = RuntimeGroup[['revenue','budget']].mean()
    RuntimeRangeDF = RuntimeRangeDF.reset_index()
# Get Profit ratio and count variables in DF
    RuntimeRangeDF['profitratio']= round(RuntimeRangeDF['revenue'] / RuntimeRangeDF['budget'],2)
    RuntimeRangeDF['count']=RuntimeMovieCount['budget']

# Display DF of runtime bins
    print(RuntimeRangeDF)
    # Plot the runtime bins with a scatter plot of profitratio, and display count as bubbles.

# Obtain the x and y coordinates for each of the three city types
    RuntimeProfitRatio = RuntimeRangeDF['profitratio']
    RuntimeBinCount = RuntimeRangeDF['count']
# Y axis will plot Average Fare ($), this is also last column in DF
#xAxis = runtime_bins
#xAxis2 = RidesSuburbanDF['Total Rides Per City']
#xAxis3 = RidesRuralDF['Total Rides Per City']

# Build the scatter plots for each city types
    plt.scatter(runtime_labels, RuntimeProfitRatio, marker="o", facecolors="coral", s=RuntimeBinCount**1.1, edgecolors="black", alpha=0.75, label = '')

# Incorporate the other graph properties
    plt.xlabel('Runtime Ranges')
    plt.ylabel('Profit Ratio')
    plt.title('Profit Ratio for Runtime ranges')
    plt.ylim(min(RuntimeProfitRatio)-0.25,max(RuntimeProfitRatio)+0.25)

# Create a legend
#plt.legend(title = 'City Types')
#plt.text(56,30,'Note: Circle size correlates with driver size per city',horizontalalignment='center',verticalalignment='center')
# Incorporate a text label regarding circle size

# Save Figure
    plt.grid()
    plt.tight_layout()
    plt.savefig("output/ProfitRatio_Runtime.png")
    plt.show()